In [1]:
# importations
using Printf
import DelimitedFiles: readdlm
using PyPlot ; const plt = PyPlot
using PyCall: pyimport
mpl = pyimport("matplotlib")
mpl.use("Qt5Agg")

In [2]:
"""
ξi, ωi, dhij = getGLL(N, dir="../gll_quad/") 
"""
function getGLL(N, dir="../gll_quadrature/")
    datafile = @sprintf("%sgll_%02d.tab",dir, N)
    data = readdlm(datafile, Float64)
    return data[1,:], data[2,:], data[3:end,:] 
end

getGLL

In [3]:
# définition système

# maillage
L = 500.0 # longueur domaine
E = 60 # nombre d'éléments
nodes = collect(range(0.0,L,length=E+1)) # maillage (régulier)

# transformation élément à élément de référence [-1,1]
le = L/E # taille des éléments
Je = le/2.0 # jacobien de [-1,1] à [e*le,(e+1)*le]
iJe = 2.0/le

# passage d'indexage (e,i) élément-point à indexage local (points dédoublés) et global (pas de points dédoublés)
localIdx(e,i) = (e-1)*(N+1) + i
globalIdx(e,i) = (e-1)*N + i
iglobalIdx(α) = α==1 ? (1,1) : (div(α-1,N)+1, α-div(α-1,N)*N)

# propriétés du milieu (constantes)
ρ = 1000.0
μ = 1e9

# résolution temporelle
Nit = 2000
T = 1.0
Δt = T/Nit

# terme de source : point  source
xs = L/2.0
t0 = T/10.0
f0 = 50
f(t) = (4*(t-t0)^2*f0^4 - 2*f0^2)*exp(-(t-t0)^2*f0^2)

# conditions aux limites
# surface libre (rien à faire)

# règle de quadrature
N = 5 # degré 
ξi, ωi, dhij = getGLL(N+1) # GLL points + lagrange interpolants derivative at theses points

# calcule position de tous les points où est calculé la solution
points = fill(0.0, E*N+1)
for α=1:E*N+1
    e,i = iglobalIdx(α)
    points[α] = le*(e + (ξi[i]+1)/2)
end

# source position
fidx = findfirst(x-> x>xs, points)
;

In [4]:
# plot des points de quadrature
fig, ax = plt.subplots()
for i=3:20
    ax.plot(getGLL(i)[1], [i for j=1:i], ".k")
end
ax.hlines(N+1, -1, 1)
plt.show()

In [5]:
# plot de la fonction source
time = collect(range(0,T, length=500))
plt.plot(time, f.(time))
plt.show()
;

In [6]:
# CONSTRUCTION DES MATRICES LOCALES

ML = fill(0.0, (E*(N+1), E*(N+1)))
KL = fill(0.0, (E*(N+1), E*(N+1)))

# MATRICE DE MASSE

for e=1:E, i=1:N+1
    α = localIdx(e,i)
    ML[α,α] = ωi[i] * ρ * Je
end

# STIFFNESS MATRIX

for e=1:E, i=1:N+1, j=1:N+1, k=1:N+1
    αi, αj = localIdx(e,i), localIdx(e,j)
    KL[αi, αj] += ωi[k] * μ * dhij[j,k] * dhij[i,k] * iJe
end

In [7]:
# ASSEMBLAGE EN MATRICES GLOBALES

Q = fill(0.0, (E*N+1, E*(N+1)))

for e=1:E, i=1:N+1
    α = localIdx(e,i)
    β = globalIdx(e,i)
    Q[β,α] = 1.0
end

M = Q*ML*Q'
K = Q*KL*Q'
iM = inv(M)
;

In [8]:
# CALCUL DYNAMIQUE

U = fill(0.0, (Nit,E*N+1))
F = fill(0.0, E*N+1)

println("Courant number : ", sqrt(μ/ρ)*Δt/(points[2]-points[1]))

for i=3:Nit
    F[fidx] = f(i*Δt)
    U[i,:] = 2 .*U[i-1,:] .- U[i-2,:] + Δt^2 .* (iM*(F-K*U[i-1,:]))
end

Courant number : 0.510758541146825


In [9]:
function plot_field(X, U)

    Nt = size(U)[1]
 
    fig, ax = plt.subplots()
    ax.set_title("1D seismic wave propagation (SEM)")
    
    umax = maximum(U)
    ax.set_ylim([-umax,umax])

    ax.set_xlabel("Depth (m)")
    ax.set_ylabel("Displacement (m)")

    i = 1
    line, = ax.plot(X, U[i,:])

    ax.grid()
    
    i_update = 3
        
    updatefig(i) = line.set_ydata(U[1+i*i_update,:])

    anim = mpl.animation.FuncAnimation(fig, updatefig, interval=30, frames=div(Nt,i_update))

    anim.event_source.start()

    plt.show()
end



plot_field (generic function with 1 method)

In [10]:
plot_field(points, U)
;